## Demo

In [1]:
import pandas as pd
from core.retriever import Retriever

r = Retriever(
    csv_path="data/data.csv",
    model_name=".cache/embeddings/Qwen3-Embedding-0.6B",  retrieve_k=64,
    reranker_name=".cache/embeddings/bge-reranker-v2-m3", top_k=8,  # optional
    rebuild_index=True
)

r.load()

query = "hicup"
nodes = r.search(query)

pd.DataFrame([{"id": n.metadata["id"], "text": n.text, "score": n.score} for n in nodes])

Table data_qwen3-embedding-0.6b doesn't exist yet. Please add some data to create it.


Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

[2026-02-23T09:00:19Z WARN  lance::dataset::write::insert] No existing dataset at /home/fame/Workspace/Retriever/.cache/lancedb/data_qwen3-embedding-0.6b.lance, it will be created


Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1301 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


,id,text,score
0,article_672c869996a92aa406c1e0ea-005,อาการสะอึกซ้ำซาก (Recurrent hiccups) คืออาการส...,0.038727
1,article_67d116660d1de5e6544d3fe9-000,วัคซีนฮิบคือวัคซีนที่ป้องกันการติดเชื้อแบคทีเร...,-1.722656
2,article_67d116660d1de5e6544d3fe9-005,วัคซีนฮิบใช้ป้องกันโรคเยื่อหุ้มสมองอักเสบ ปอดบ...,-3.083984


In [2]:
len(nodes)

8

## Implementation

### Retrival

In [3]:
import pandas as pd
from llama_index.core.schema import TextNode
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, VectorStoreIndex, StorageContext


df = pd.read_csv("data/demo.csv")

nodes = [
    TextNode(text=row["text"],metadata={"id": row["id"]})
        for _, row in df.iterrows() if not pd.isna(row['text'])
]

## Embedding
Settings.embed_model = HuggingFaceEmbedding(model_name="Qwen/Qwen3-Embedding-0.6B", device="cuda", normalize=True,)

## Vector DB
vector_store = LanceDBVectorStore(uri="./.cache/lancedb", table_name="embeddings")
storage = StorageContext.from_defaults(vector_store=vector_store)

## Retriever
index = VectorStoreIndex(nodes, storage_context=storage, show_progress=True)
index.storage_context.persist(persist_dir='.cache')
retriever = index.as_retriever(similarity_top_k=64)


## Inference
query = "hicup"
# Retrieve
nodes = retriever.retrieve(query)

pd.DataFrame([{"id":o.metadata['id'], "text": o.text, "score": o.score} for o in nodes])

Table embeddings doesn't exist yet. Please add some data to create it.


Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

[2026-02-23T09:08:51Z WARN  lance::dataset::write::insert] No existing dataset at /home/fame/Workspace/Retriever/.cache/lancedb/embeddings.lance, it will be created


,id,text,score
0,article_67f7a1c39b542b3157bd5048-000,โรคแพนิคหรือโรคตื่นตระหนกเป็นภาวะที่มีอาการวิต...,0.186149
1,article_67f7a1c39b542b3157bd5048-002,อาการของโรคแพนิคแบ่งเป็นสองกลุ่มคือ อาการทางร่...,0.172363
2,article_67f7a1c39b542b3157bd5048-001,สาเหตุหลักของโรคแพนิคคือการทำงานผิดปกติของสมอง...,0.170076


### Reranker

In [4]:
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

## Reranker
reranker = FlagEmbeddingReranker(model="BAAI/bge-reranker-v2-m3", top_n=3, use_fp16=True)

# Ranking
out = reranker.postprocess_nodes(nodes, query_str=query)
pd.DataFrame([{"id":o.metadata['id'], "text": o.text, "score": o.score} for o in out])

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


,id,text,score
0,article_67f7a1c39b542b3157bd5048-000,โรคแพนิคหรือโรคตื่นตระหนกเป็นภาวะที่มีอาการวิต...,-9.257812
1,article_67f7a1c39b542b3157bd5048-001,สาเหตุหลักของโรคแพนิคคือการทำงานผิดปกติของสมอง...,-10.343750
2,article_67f7a1c39b542b3157bd5048-002,อาการของโรคแพนิคแบ่งเป็นสองกลุ่มคือ อาการทางร่...,-10.507812
